In [83]:
#grab the required packages
#%pip install pandas
#%pip install FRB

from fred import Fred
#https://frb.readthedocs.io/en/latest/usage.html#id1

import pandas as pd
import os
import json

#TODO: create JSON HOF to wrap every call in the conversion to json observations =  json.dumps(json.loads(fr.series.observations('MORTGAGE30US'))['observations'])

#FRED API KEY LOAD
API_KEY = os.environ.get('FRED_API_KEY')

#INIT THE CLIENT
fr = Fred(api_key=API_KEY,response_type='json')



In [84]:
#parent level category
res = fr.category.details(97)

#list all metrics available in the housing category
print(fr.category.children(97))

#housing starts category
print(fr.category.details(32302))


#list all series available in this category
res = json.loads(fr.category.series(32302))
#target the seriess object and select housing starts (the first in the list)
print(res["seriess"][0])



{"categories":[{"id":32300,"name":"Housing Units Authorized by Building Permits","parent_id":97},{"id":32301,"name":"Housing Units Authorized, But Not Yet Started","parent_id":97},{"id":32302,"name":"Housing Starts","parent_id":97},{"id":32303,"name":"Housing Units Under Construction","parent_id":97},{"id":32304,"name":"Housing Units Completed","parent_id":97},{"id":32426,"name":"New Residential Sales","parent_id":97},{"id":33949,"name":"Manufactured Housing","parent_id":97}]}
{"categories":[{"id":32302,"name":"Housing Starts","parent_id":97}]}
{'id': 'HOUST', 'realtime_start': '2023-01-18', 'realtime_end': '2023-01-18', 'title': 'New Privately-Owned Housing Units Started: Total Units', 'observation_start': '1959-01-01', 'observation_end': '2022-11-01', 'frequency': 'Monthly', 'frequency_short': 'M', 'units': 'Thousands of Units', 'units_short': 'Thous. of Units', 'seasonal_adjustment': 'Seasonally Adjusted Annual Rate', 'seasonal_adjustment_short': 'SAAR', 'last_updated': '2022-12-20 

In [97]:
#call the historical housing starts series
observations = json.dumps(json.loads(fr.series.observations('HOUST'))['observations'])
HOUST_DF = pd.read_json(observations)
#convert date strings to proper datetimes
HOUST_DF['date'] = pd.to_datetime(HOUST_DF['date'])
HOUST_DF.columns = ['realtime_start', 'realtime_end', 'date', 'HOUSING_STARTS']
HOUST_DF.drop(axis=1, columns=['realtime_start','realtime_end'], inplace=True)
HOUST_DF.set_index("date", inplace=True)
HOUST_DF = HOUST_DF.resample('D').ffill() #this forward fills the previous value up until a new value exists

print(HOUST_DF.head())

            HOUSING_STARTS
date                      
1959-01-01            1657
1959-01-02            1657
1959-01-03            1657
1959-01-04            1657
1959-01-05            1657


In [99]:
#call historical mortgage rates
observations = json.dumps(json.loads(fr.series.observations('MORTGAGE30US'))['observations'])
MTG_DF = pd.read_json(observations) 
MTG_DF.columns = ['realtime_start', 'realtime_end', 'date', 'MTG_RATE']

#convert date strings to proper datetimes
MTG_DF['date'] = pd.to_datetime(MTG_DF['date'])
MTG_DF.columns = ['realtime_start', 'realtime_end', 'date', 'HOUSING_STARTS']
MTG_DF.drop(axis=1, columns=['realtime_start','realtime_end'], inplace=True)
MTG_DF.set_index('date', inplace=True)
# using the resample method
# https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.fillna.html
MTG_DF = MTG_DF.resample('D').ffill() #this forward fills the previous value up until a new value exists

#MTG_DF = MTG_DF.resample('M').ffill() #monthly downsample
print(MTG_DF.head())

            HOUSING_STARTS
date                      
1971-04-02            7.33
1971-04-03            7.33
1971-04-04            7.33
1971-04-05            7.33
1971-04-06            7.33


In [105]:
#merge dataframes
MAIN_FRAME = pd.merge(MTG_DF, HOUST_DF, left_index=True, right_index=True)

print(MAIN_FRAME)


MergeError: Must pass right_on or right_index=True

In [88]:
#interpolate the less frequent observation to match the more frequent https://stackoverflow.com/questions/29612705/converting-pandas-dataframe-from-monthly-to-daily
START_DATE = '2000-01-01'
END_DATE = '2022-01-01'
DATE_RANGE =  pd.date_range(START_DATE, END_DATE, freq='D')
#print(DATE_RANGE)

# pseudocode - grab the min date from all series, grab the max date from all series, past the values in matching dates, then interpolate